In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

file_path = '../../data/school_health_preprocessed.csv'

# 데이터 로드
try:
    data = pd.read_csv(file_path, encoding='utf-8')
    print("데이터 로드 성공!")
except FileNotFoundError:
    print("파일을 찾을 수 없습니다. 경로를 확인하세요.")

# 결측치 제거
cleaned_data = data.dropna(subset=['BMI', '하루30분이상운동'])
print(f"Cleaned data size: {cleaned_data.shape}")

# 성별로 데이터 분리
male_data = cleaned_data[cleaned_data['성별'] == '남'].copy()
female_data = cleaned_data[cleaned_data['성별'] == '여'].copy()

# 로그 변환 (음수 및 0 제거)
male_data = male_data[male_data['BMI'] > 0].copy()
female_data = female_data[female_data['BMI'] > 0].copy()
male_data['log_BMI'] = np.log(male_data['BMI'])
female_data['log_BMI'] = np.log(female_data['BMI'])

# 남학생 OLS 회귀 분석
print("\n[OLS 회귀 - 남학생]")
male_X = sm.add_constant(male_data['하루30분이상운동'])  # 상수항 추가
male_y = male_data['log_BMI']
male_model = sm.OLS(male_y, male_X).fit()
print(male_model.summary())



데이터 로드 성공!
Cleaned data size: (49790, 111)

[OLS 회귀 - 남학생]
                            OLS Regression Results                            
Dep. Variable:                log_BMI   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     5.171
Date:                Tue, 10 Dec 2024   Prob (F-statistic):             0.0230
Time:                        03:16:24   Log-Likelihood:                 8692.1
No. Observations:               25033   AIC:                        -1.738e+04
Df Residuals:                   25031   BIC:                        -1.736e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

In [3]:
# 여학생 OLS 회귀 분석
print("\n[OLS 회귀 - 여학생]")
female_X = sm.add_constant(female_data['하루30분이상운동'])  # 상수항 추가
female_y = female_data['log_BMI']
female_model = sm.OLS(female_y, female_X).fit()
print(female_model.summary())



[OLS 회귀 - 여학생]
                            OLS Regression Results                            
Dep. Variable:                log_BMI   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     82.27
Date:                Tue, 10 Dec 2024   Prob (F-statistic):           1.27e-19
Time:                        03:16:26   Log-Likelihood:                 10749.
No. Observations:               24757   AIC:                        -2.149e+04
Df Residuals:                   24755   BIC:                        -2.148e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0242      0.002   1

# OLS 회귀 분석 결과

## 1. 남학생 회귀 분석 결과
### **모델 적합성**
- **R-squared (설명력)**: 0.000
  - 독립 변수 `하루30분이상운동`이 종속 변수 `log_BMI`의 변동성을 거의 설명하지 못함.
  - 운동 빈도가 BMI에 미치는 영향은 매우 미미함.
- **F-statistic**: 5.171, **p-value**: 0.023
  - 모델 전체의 유의성을 검정하는 F-test에서 p-value가 0.05보다 작으므로, `하루30분이상운동`이 `log_BMI`에 미치는 영향이 통계적으로 유의미함.

### **회귀 계수**
- **상수항 (const)**: 3.0847
  - `하루30분이상운동` 값이 0일 때의 `log_BMI` 예측값.
- **`하루30분이상운동`**: 0.0023
  - `하루30분이상운동`이 1 증가할 때, `log_BMI`가 평균적으로 0.0023만큼 증가.
  - 효과 크기(coef)가 매우 작아 실제 영향은 미미함.

### **잔차 분석**
- **Omnibus, Jarque-Bera (JB) Test**:
  - Prob(Omnibus): 0.000, Prob(JB): 8.61e-82
  - 잔차가 정규성을 따르지 않음을 나타냄.
- **Durbin-Watson**: 1.928
  - 잔차 간 자기 상관성이 거의 없음.

### **결론**
- 운동 빈도(`하루30분이상운동`)는 통계적으로 `log_BMI`에 유의미한 영향을 미치지만, 실제 영향력은 매우 낮음.
- R-squared 값이 0.000으로 BMI를 설명하기엔 운동 외 다른 요인을 고려해야 함.

---

## 2. 여학생 회귀 분석 결과
### **모델 적합성**
- **R-squared (설명력)**: 0.003
  - 독립 변수 `하루30분이상운동`이 `log_BMI`의 변동성을 일부 설명.
- **F-statistic**: 82.27, **p-value**: 1.27e-19
  - `하루30분이상운동`이 `log_BMI`에 미치는 영향이 통계적으로 매우 유의미함.

### **회귀 계수**
- **상수항 (const)**: 3.0242
  - `하루30분이상운동` 값이 0일 때의 `log_BMI` 예측값.
- **`하루30분이상운동`**: 0.0098
  - `하루30분이상운동`이 1 증가할 때, `log_BMI`가 평균적으로 0.0098만큼 증가.
  - 남학생에 비해 여학생의 효과 크기가 더 큼.

### **잔차 분석**
- **Omnibus, Jarque-Bera (JB) Test**:
  - Prob(Omnibus): 0.000, Prob(JB): 1.69e-92
  - 잔차가 정규성을 따르지 않음을 나타냄.
- **Durbin-Watson**: 1.901
  - 잔차 간 자기 상관성이 거의 없음.

  
